### ChatSession.history
### 透過 ChatSession.history,記錄每一個聊天階段所保留的值
- ChatSession.history屬性儲存依時間順序的user記錄和model的記錄
- 每一輪的對話儲存於ganai.protos.Content實體內,此實體內的資訊有:
	- Role:代表是user或者是model
	- Parts:genai.protos.Part
		- Text:代表純文字訊息
		- Function Call(genai.protos.FunctionCall):一個model的請求,有包含要執行的function和參數
		- Function Response(genai.protos.FunctionResponse):執行funciton完成後,使用者傳出的結果

In [1]:
import os
import google.generativeai as genai

def add(a:float, b:float)->float:
    '''
    傳出a + b.
    '''
    return a + b

def subtract(a:float, b:float)->float:
    '''
    傳出a - b.
    '''
    return a - b

def multiply(a:float, b:float) -> float:
    '''
    傳出a * b.
    '''

    return a * b

def divide(a:float, b:float):
    '''
    傳出 a / b.
    '''

    return a / b

genai.configure(api_key=os.environ['GEMINI_API_KEY'])
model = genai.GenerativeModel(
    model_name = 'gemini-2.0-flash-exp',
    tools = [add, multiply, divide, subtract]
)

chat = model.start_chat(enable_automatic_function_calling=True)

response = chat.send_message(
    "I have 57 cats, each owns 44 mittens, how many mittens is that in total?"
)
response.text

for content in chat.history:
    print(content.role, '->', [type(part).to_dict(part) for part in content.parts])
    print("-" * 80)

user -> [{'text': 'I have 57 cats, each owns 44 mittens, how many mittens is that in total?'}]
--------------------------------------------------------------------------------
model -> [{'function_call': {'name': 'multiply', 'args': {'a': 57.0, 'b': 44.0}}}]
--------------------------------------------------------------------------------
user -> [{'function_response': {'name': 'multiply', 'response': {'result': 2508.0}}}]
--------------------------------------------------------------------------------
model -> [{'text': 'There are 2508 mittens in total.\n'}]
--------------------------------------------------------------------------------


![](./images/pic1.png)

> user -> text -> 提問 

> model -> function_calling -> 得到funciton名稱和引數值

> user -> function_response -> 呼叫該function() (自動呼叫)

> model -> text -> 收到function傳回來的值整合至model的輸出